# Ensure notebooks can execute in parallel

This notebook uses a file system based "lock" to assert that two instances of the notebook kernel will run in parallel. Each instance writes to a file in a temporary directory, and then tries to read the other file from
the temporary directory, so that running them in sequence will fail, but running them in parallel will succeed.

Two notebooks are launched, each which sets the `this_notebook` variable. One notebook is set to `this_notebook = 'A'` and the other `this_notebook = 'B'`.

In [ ]:
import os
import os.path
import tempfile
import time

In [ ]:
# the variable this_notebook is injectected in a cell above by the test framework.
this_notebook = 'A'
other_notebook = 'B'
directory = os.environ['NBEXECUTE_TEST_PARALLEL_TMPDIR']
with open(os.path.join(directory, 'test_file_{}.txt'.format(this_notebook)), 'w') as f:
    f.write('Hello from {}'.format(this_notebook))

In [ ]:
start = time.time()
timeout = 5
end = start + timeout
target_file = os.path.join(directory, 'test_file_{}.txt'.format(other_notebook))
while time.time() < end:
    time.sleep(0.1)
    if os.path.exists(target_file):
        with open(target_file, 'r') as f:
            text = f.read()
        if text == 'Hello from {}'.format(other_notebook):
            break
else:
    assert False, "Timed out – didn't get a message from {}".format(other_notebook)